## Постановка задачи
Загрузим данные, приведем их к числовым, заполним пропуски, нормализуем данные и оптимизируем память.

Разделим выборку на обучающую/проверочную в соотношении 80/20.

Сформируем параллельный ансамбль из логистической регрессии, SVM, случайного леса и LightGBM. Используем лучшие гиперпараметры, подобранные ранее. Итоговое решение рассчитаем на основании весов (вероятностей).

Проведем предсказание и проверим качество через каппа-метрику.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

© ITtensive, 2020

In [1]:
GRAIN = 11
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn import preprocessing
from etl_utils import reduce_mem_usage


data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")

data['Product_Info_2_1'] = data['Product_Info_2'].str.slice(0, 1)
data['Product_Info_2_2'] = pd.to_numeric(data['Product_Info_2'].str.slice(1, 2))
data = data.drop('Product_Info_2', axis='columns')

onehot_df = pd.get_dummies(data['Product_Info_2_1'])
onehot_df.columns = ['Product_Info_2_1' + column for column in onehot_df.columns]
data = pd.merge(left=data, right=onehot_df, left_index=True, right_index=True).drop('Product_Info_2_1', axis=1).fillna(-1)
del onehot_df

data['Response'] = data['Response'] - 1

### Набор столбцов для расчета

In [2]:
columns_groups = ['Insurance_History', 'InsurеdInfo', 'Medical_Keyword', 'Family_Hist', 'Medical_History', 'Product_Info']
columns = ['Wt', 'Ht', 'Ins_Age', 'BMI']
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print(columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Нормализация данных и Разделение данных

In [3]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(data[columns]))
columns_transformed = data_transformed.columns
data_transformed['Response'] = data['Response']
data_transformed = reduce_mem_usage(data_transformed)

data_train, data_test = train_test_split(data_transformed, test_size=.2, random_state=GRAIN)
print(data_train.shape, data_test.shape)

Потребление памяти меньше на 40.49 Мб (-75.1%)
(47504, 119) (11877, 119)


### Построение базовых моделей

In [4]:
x = data_train[columns_transformed]

SVM

In [5]:
model_svm = SVC(kernel='linear', probability=True, max_iter=10000).fit(x, data_train['Response'])

C:\Users\soede\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Логистическая регрессия

In [9]:
model_logr = LogisticRegression(max_iter=1000).fit(x, data_train['Response'])

Случайный лес

In [11]:
model_rf = RandomForestClassifier(
    random_state=GRAIN,
    n_estimators=76,
    max_depth=17,
    max_features=27,
    min_samples_leaf=20
).fit(x, data_train['Response'])

LightGBM, используем multiclass

In [12]:
model_lgb = lgb.LGBMRegressor(
    random_state=GRAIN, max_depth=18, min_child_samples=18, num_leaves=75, n_estimators=1000,
    objective="multiclass", num_class=8
).fit(x, data_train['Response'])

### Расчет предказаний
Кроме непосредственно значений дополнительно посчитаем вероятности совпадения с тем или иным классом

In [13]:
x_test = data_test[columns_transformed]

In [14]:
data_test_svm_proba = pd.DataFrame(model_svm.predict_proba(x_test))

In [15]:
data_test_logr_proba = pd.DataFrame(model_logr.predict_proba(x_test))

In [16]:
data_test_rf_proba = pd.DataFrame(model_rf.predict_proba(x_test))

In [17]:
data_test_lgb = pd.DataFrame(model_lgb.predict(x_test))

Несколько вариантов ансамблей с голосованием (выбор класса выполняется для каждого кортежа по отдельности):
* "Мягкое" голосование (в том числе, с определенными весами): суммируются вероятности каждого класса среди всех оценок, выбирается наибольшее.
* "Жесткое" (мажоритарное) голосование: выбирается самый популярный класс среди моделей (число моделей должно быть нечетным).
* Отсечение: из вероятностей моделей выбирается только наиболее значимые, например, больше 0.3.
* Экспертное голосование: вес оценки эксперта зависит от кортежа данных и самого класса (например, если определенная модель предсказывает определенный класс точнее других).

Здесь используем "мягкое" голосование, для этого необходимо рассчитать вероятности всех класса для каждого кортежа данных.

In [18]:
def vote_class(x):
    return np.argmax(x.values)

In [52]:
data_test_proba = data_test_svm_proba.copy()
for i in range(0, data_test_proba.shape[1]):
    data_test_proba[i] *= .2
    data_test_proba[i] += data_test_logr_proba[i]
    data_test_proba[i] += data_test_rf_proba[i]
    data_test_proba[i] += 50 * data_test_lgb[i]
    
data_test_proba['target'] = data_test_proba.apply(vote_class, axis='columns')

### Оценка ансамбля
Рассчитаем оценку взвешенного предсказания 4 моделей

Кластеризация дает 0.192, kNN(100) - 0.3, лог. регрессия - 0.512/0.496, SVM - 0.95, реш. дерево - 0.3, случайный лес - 0.487, XGBoost - 0.534, градиентный бустинг - 0.545, LightGBM - 0.551, CatBoost - 0.543

In [53]:
print(
    "Ансамбль классификации:",
    round(cohen_kappa_score(data_test_proba["target"], data_test["Response"], weights="quadratic"), 3)
)

Ансамбль классификации: 0.55


### Матрица неточностей

In [28]:
print("Ансамбль классификации\n", confusion_matrix(data_test_proba["target"], data_test["Response"]))

Ансамбль классификации
 [[ 273  147   27   24   57  112   44   28]
 [ 204  338   24   13  101  109   35   17]
 [  14   26   61   16    1    0    0    0]
 [  36   25   52  205    0    2    0    1]
 [  94  131   10    0  570  106   10   10]
 [ 244  268   20   21  192 1215  277  164]
 [ 122  117    3    3   53  245  658  161]
 [ 251  255    5   34   90  447  538 3571]]


### Само-проверка
Проверим, насколько ансамбль хорошо предсказывает обучающие данные

In [55]:
data_train_svm = pd.DataFrame(model_svm.predict_proba(x))
data_train_logr = pd.DataFrame(model_logr.predict_proba(x))
data_train_rf = pd.DataFrame(model_rf.predict_proba(x))
data_train_lgb = pd.DataFrame(model_lgb.predict(x))

for i in range(0, data_train_svm.shape[1]):
    data_train_svm[i] *= .2
    data_train_svm[i] += data_train_logr[i]
    data_train_svm[i] += data_train_rf[i]
    data_train_svm[i] += 50 * data_train_lgb[i]
    
target = data_train_svm.apply(vote_class, axis='columns')
print("Результат:", round(cohen_kappa_score(target, data_train["Response"], weights="quadratic"), 3))
print(confusion_matrix(target, data_train["Response"]))

Результат: 1.0
[[ 4969     0     0     0     0     0     0     0]
 [    0  5245     0     0     0     0     0     0]
 [    0     0   811     0     0     0     0     0]
 [    0     0     0  1112     0     0     0     0]
 [    0     0     0     0  4368     0     0     0]
 [    0     0     0     0     0  8996     0     0]
 [    0     0     0     0     0     0  6463     0]
 [    0     0     0     0     0     1     2 15537]]
